In [1]:
sheet_url = "https://docs.google.com/spreadsheets/d/1B9AZ_VwCjJlSSO44CZg71vx30NafVvVrdPgzbLS9ZxU/edit#gid=2075537893"

año = '2022'

In [2]:
#Transformación de Data
import pandas as pd
import numpy as np
import mysql.connector as connection
import warnings
warnings.filterwarnings('ignore')


url_1 = sheet_url.replace('/edit#gid=', '/export?format=csv&gid=')
df = pd.read_csv(url_1)

df = df[["FECHA", "PERIODO","TIPO ACOMPAÑAMIENTO","RESPONSABLES DE COORD CON LA UO","UO (LÍDERA)","ARTICULACIÓN CON OTRAS UO","HORA","DIRECTIVOS/JEFES (SI/NO)","TIPO","COD DE AT","COD SISGED","TITULO","ALCANCE","SESIONES","REGIONES POR FECHA","REGIONES DE LA ATA","ENFASIS TÉMATICO DE LA AT","N.° DE CONECTADOS","TIPO DE AT (VIRTUAL/ PRESENCIAL)","DRE/GRE, UGEL","PERFIL","NUDO","OBJETIVO","PRODUCTO","PROCESOS","PROCESOS 0","PROCESOS 1","NORMA RELACIONADA"]]

# PARCHE PERFIL
df["PERFILX"] = np.where(df["COD SISGED"] == "ATE03716" , "ENCARGADO DE TRANSPORTE, ESPECIALISTA DE PLANIFICACIÓN, ESPECIALISTA DE ADMINISTRACIÓN, ESPECIALISTA DE PATRIMONIO, ESPECIALISTA DE ATENCIÓN AL USUARIO, ESPECIALISTA DE ESCALAFÓN, ESPECIALISTA EN ESTADÍSTICA, ESPECIALISTA DE PERSONAL/RRHH, ESPECIALISTA EN MONITOREO PEDAGÓGICO, ESPECIALISTA EN INFRAESTRUCTURA, ESPECIALISTA EN INFORMÁTICA, ESPECIALISTA EN ALMACÉN", df["PERFIL"])
df["PERFILX"] = np.where(df["COD SISGED"] == "ATE03873" , "DIRECTOR DRE, DIRECTIVOS DGI/DGA, ESPECIALISTAS FUM, DIRECTOR UGEL, JEFE GESTIÓN INSTITUCIONAL Y ADMINISTRATIVO, ESPECIALISTAS DE PROCESOS ADMINISTRATIVOS", df["PERFILX"])

#PARCHE TIPO ACOMP
df["TIPO ACOMPAÑAMIENTO"] = np.where(df["COD SISGED"] == "ATE03873" , "MIXTA", df["TIPO ACOMPAÑAMIENTO"])

# Limpieza sin fecha
df["FECHA"].fillna('', inplace=True)
df = df.drop(df[df["FECHA"]==""].index)
df["COD SISGED"].fillna('', inplace=True)
df["COD DE AT"].fillna('', inplace=True)
df1 = df[df["TIPO"]=="AT"]
df3 = df1.drop(df1[df1["COD SISGED"]==""].index)
df3["kpi_titulo"] = df3["COD SISGED"].map(str)+df3["TITULO"].map(str)
df3["kpi_codat"] = df3["COD SISGED"].map(str)+df3["COD DE AT"].map(str)
df3["kpi_3"] = df3["COD SISGED"].map(str)+df3["TIPO ACOMPAÑAMIENTO"].map(str)
df3["kpi_4"] = df3["COD SISGED"].map(str)+df3["UO (LÍDERA)"].map(str)
df3["kpi_5"] = df3["COD SISGED"].map(str)+df3["ENFASIS TÉMATICO DE LA AT"].map(str)
df3["kpi_6"] = df3["COD SISGED"].map(str)+df3["NUDO"].map(str)
df3["kpi_7"] = df3["COD SISGED"].map(str)+df3["OBJETIVO"].map(str)
df3["kpi_8"] = df3["COD SISGED"].map(str)+df3["PROCESOS"].map(str)
df3["kpi_9"] = df3["COD SISGED"].map(str)+df3["PROCESOS 0"].map(str)
df3["kpi_10"] = df3["COD SISGED"].map(str)+df3["PROCESOS 1"].map(str)
df3["kpi_11"] = df3["COD SISGED"].map(str)+df3["ALCANCE"].map(str)
df3["kpi_12"] = df3["COD SISGED"].map(str)+df3["PERFILX"].map(str)
df3["kpi_13"] = df3["COD SISGED"].map(str)+df3["PRODUCTO"].map(str)
df3["kpi_14"] = df3["COD SISGED"].map(str)+df3["DIRECTIVOS/JEFES (SI/NO)"].map(str)
df3["kpi_15"] = df3["COD SISGED"].map(str)+df3["ARTICULACIÓN CON OTRAS UO"].map(str)

In [ ]:
#Crear DataFrame para SQL
df3["FECHA1"] = df["FECHA"].str.replace('/','')
df3["FECHA1"]= df3['FECHA1'].astype(str).str.zfill(8)
df3["FECHA2"] = pd.to_datetime(df3["FECHA1"],format='%d%m%Y')

df4 = pd.DataFrame(df3.groupby(["COD SISGED"])["FECHA2"].max())
df4['COD SISGED'] = df4.index
df4["llave1"] = df4['COD SISGED']

df3["TITULO"]=df3["TITULO"].str.upper()
df3["ALCANCE"]=df3["ALCANCE"].str.upper()
df3["ARTICULACIÓN CON OTRAS UO"]=df3["ARTICULACIÓN CON OTRAS UO"].str.upper()
df3["ENFASIS TÉMATICO DE LA AT"]=df3["ENFASIS TÉMATICO DE LA AT"].str.upper()
df3["PERFILX"]=df3["PERFILX"].str.upper()
df3["NUDO"]=df3["NUDO"].str.upper()
df3["OBJETIVO"]=df3["OBJETIVO"].str.upper()
df3["PRODUCTO"]=df3["PRODUCTO"].str.upper()
df3["PROCESOS"]=df3["PROCESOS"].str.upper()
df3["PROCESOS 0"]=df3["PROCESOS 0"].str.upper()
df3["PROCESOS 1"]=df3["PROCESOS 1"].str.upper()

df5=df3.drop(['FECHA'], axis=1)
df5=df5.drop(['PERIODO'], axis=1)
df5=df5.drop(['RESPONSABLES DE COORD CON LA UO'], axis=1)
df5=df5.drop(['REGIONES POR FECHA'], axis=1)
df5=df5.drop(['REGIONES DE LA ATA'], axis=1)
df5=df5.drop(['SESIONES'], axis=1)
df5=df5.drop(['N.° DE CONECTADOS'], axis=1)
df5=df5.drop(['TIPO DE AT (VIRTUAL/ PRESENCIAL)'], axis=1)
df5=df5.drop(['DRE/GRE, UGEL'], axis=1)
df5=df5.drop(['NORMA RELACIONADA'], axis=1)
df5=df5.drop(['PERFIL'], axis=1)
df5=df5.drop(['kpi_titulo'], axis=1)
df5=df5.drop(['kpi_codat'], axis=1)
df5=df5.drop(['kpi_3'], axis=1)
df5=df5.drop(['kpi_4'], axis=1)
df5=df5.drop(['kpi_5'], axis=1)
df5=df5.drop(['kpi_6'], axis=1)
df5=df5.drop(['kpi_7'], axis=1)
df5=df5.drop(['kpi_8'], axis=1)
df5=df5.drop(['kpi_9'], axis=1)
df5=df5.drop(['kpi_10'], axis=1)
df5=df5.drop(['kpi_11'], axis=1)
df5=df5.drop(['kpi_12'], axis=1)
df5=df5.drop(['kpi_13'], axis=1)
df5=df5.drop(['kpi_14'], axis=1)
df5=df5.drop(['kpi_15'], axis=1)
df5=df5.drop(['FECHA1'], axis=1)
df5=df5.drop(['FECHA2'], axis=1)
df5=df5.drop(['HORA'], axis=1)

df6= df5.drop_duplicates()
df6["llave1"] = df6["COD SISGED"]
df6 = df6.merge(df4, on="llave1", how="left")
df6['AÑO'] = pd.DatetimeIndex(df6['FECHA2']).year
df6['MES'] = pd.DatetimeIndex(df6['FECHA2']).month
df6['MES2'] = pd.to_datetime(df6['MES'], format='%m').dt.month_name(locale = 'Spanish').str.slice(stop=3)
df6["MES"]= df6['MES'].astype(str).str.zfill(2)
df6['MESX'] = df6["MES"].map(str)+". "+df6["MES2"].map(str)

df6=df6.drop(['MES'], axis=1)
df6=df6.drop(['MES2'], axis=1)

#Crear Excel y Parche MESX de 6 AT
df6["MESX"] = np.where(df6["COD SISGED_x"]== "ATE03347", "02. Feb", df6["MESX"])
df6["MESX"] = np.where(df6["COD SISGED_x"]== "ATE03461", "03. Mar", df6["MESX"])
df6["MESX"] = np.where(df6["COD SISGED_x"]== "ATE03535", "04. Abr", df6["MESX"])
df6["MESX"] = np.where(df6["COD SISGED_x"]== "ATE03526", "04. Abr", df6["MESX"])
df6["MESX"] = np.where(df6["COD SISGED_x"]== "ATE03639", "06. Jun", df6["MESX"])
df6["MESX"] = np.where(df6["COD SISGED_x"]== "ATE03481", "06. Jun", df6["MESX"])

df6["FECHA2"] = pd.to_datetime(df6["FECHA2"]).dt.date
df6['FECHA2'] = df6['FECHA2'].astype(str)
df6["AÑO"] = df6["AÑO"].astype(str)
df6.fillna('', inplace=True)
df6["COD SISGED_x"] = df6["COD SISGED_x"].str.replace(' ','')
import openpyxl
df6.to_excel("C:/Users/difoca05/OneDrive - Ministerio de Educación/ASISTENCIA TECNICA (ATA)/Reporte_ATA_Planificador.xlsx", index=False, sheet_name='base_datos')

#Crear DataFrame para SQL2
df7 = df3
df7['HORA'] = df7['HORA'].str.replace('.',':')
df7['HORA'] = df7['HORA'].str.replace(' ','')
df7['HORA'] = df7['HORA'].str.replace(';',',')
df7['SESIONES']="S"+df7["SESIONES"].map(str)
df7['SESIONES']=df7['SESIONES'].str.replace(',',',S')
df7['REGIONES POR FECHA']=df7['REGIONES POR FECHA'].str.upper()

# sub = df7["COD SISGED"].isin(['ATE03653-2'])
# col = ["COD SISGED", "SESIONES"]
# df7.loc[sub, col]

df8 = df7[["COD SISGED","FECHA2","HORA", "SESIONES", "REGIONES POR FECHA", "DIRECTIVOS/JEFES (SI/NO)",  "ALCANCE","TIPO DE AT (VIRTUAL/ PRESENCIAL)"]].sort_values(by='FECHA2', ascending=False)

df8.fillna('', inplace=True)

df8["FECHA2"] = pd.to_datetime(df8["FECHA2"]).dt.date
df8['FECHA2'] = df8['FECHA2'].astype(str)

df8.to_excel("C:/Users/difoca05/OneDrive - Ministerio de Educación/ASISTENCIA TECNICA (ATA)/Reporte_ATA_Calendario.xlsx", index=False, sheet_name='base_datos')

In [ ]:
#Delete Planificador SISGED
import Conexion_mysql as cnx

sql1="delete from ATA_PLANIFICADOR_SISGED where año = "+ año
cnx.query(sql1)

In [ ]:
#Insert Planificador SISGED
sql2 = ''
for (rows,rs) in df6.iterrows():
                i1 = rs[6]
                i2 = rs[5]
                i3 = rs[0]
                i4 = rs[1]
                i5 = rs[7]
                i6 = rs[9]
                i7 = rs[10]
                i8 = rs[11]
                i9 = rs[13]
                i10 = rs[14]
                i11 = rs[15]
                i12 = rs[8]
                i13 = rs[16]
                i14 = rs[12]
                i15 = rs[3]
                i16 = rs[20]
                i17 = rs[21]
                text = "('"+i1+"', '"+i2+"', '"+i3+"', '"+i4+"', '"+i5+"', '"+i6+"', '"+i7+"', '"+i8+"', '"+i9+"', '"+i10+"', '"+i11+"', '"+i12+"', '"+i13+"', '"+i14+"', '"+i15+"', '"+i16+"', '"+i17+"'),"
                sql2 = sql2 + text
sql2=sql2[:-1]
sql3="insert into ata_planificador_sisged values " + sql2
cnx.query(sql3)

In [ ]:
#Delete Planificador Sesiones
sql4="delete from ATA_PLANIFICADOR_SESION where year(fecha)='" + año + "'"
cnx.query(sql4)

In [ ]:
#Insert Planificador Sesiones
sql5=''
for (rows,rs) in df8.iterrows():
                i1 = rs[0]
                i2 = rs[1]
                i3 = rs[2]
                i4 = rs[3]
                i5 = rs[4]
                i6 = rs[5]
                i7 = rs[6]
                i8 = rs[7]
                text = "('"+i1+"', '"+i2+"', '"+i3+"', '"+i4+"', '"+i5+"', '"+i6+"', '"+i7+"', '"+i8+"'),"
                sql5=sql5 + text
sql5=sql5[:-1]
sql5="insert into ATA_PLANIFICADOR_SESION (COD_SISGED, FECHA, HORA, SESION, REGION, DIRECTIVO, ALCANCE, virtual_presc) values " + sql5
cnx.query(sql5)